In [ ]:
!pip install pennylane openfermionpyscf zstd dill --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 KB 7.5 MB/s eta 0:00:00


In [ ]:
# scipy need to be upgraded in Colab to resolve issues with loading molecular data from Pennylane datasets
!pip install scipy --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 19.4 MB/s eta 0:00:00


Qubit tapering
==============

::: {.meta}
:property=\"og:description\": Learn how to taper off qubits
:property=\"og:image\":
<https://pennylane.ai/qml/_images/qubit_tapering.png>
:::

::: {.related}
tutorial\_quantum\_chemistry Building molecular Hamiltonians
tutorial\_vqe A brief overview of VQE tutorial\_givens\_rotations Givens
rotations for quantum chemistry tutorial\_adaptive\_circuits Adaptive
circuits for quantum chemistry tutorial\_differentiable\_HF
Differentiable Hartree-Fock
:::

*Authors: Utkarsh Azad and Soran Jahangiri. Posted: 16 May 2022. Last
updated: 08 Nov 2022*

The performance of variational quantum algorithms is considerably
limited by the number of qubits required to represent wave functions. In
the context of quantum chemistry, this limitation hinders the treatment
of large molecules with algorithms such as the `variational quantum
eigensolver (VQE) <tutorial_vqe>`{.interpreted-text role="doc"}. Several
approaches have been developed to reduce the qubit requirements for
quantum chemistry calculations. In this tutorial, we demonstrate the
symmetry-based qubit tapering approach which allows reducing the number
of qubits required to perform molecular quantum simulations based on the
$\mathbb{Z}_2$ symmetries present in molecular Hamiltonians .

A molecular Hamiltonian in the qubit basis can be expressed as a linear
combination of Pauli words as

$$H = \sum_{i=1}^r h_i P_i,$$

where $h_i$ is a real coefficient and $P_i$ is a tensor product of Pauli
and identity operators acting on $M$ qubits

$$P_i \in \pm \left \{ I, X, Y, Z \right \} ^ {\bigotimes M}.$$

The main idea in the symmetry-based qubit tapering approach is to find a
unitary operator $U$ that transforms $H$ to a new Hamiltonian $H'$ which
has the same eigenvalues as $H$

$$H' = U^{\dagger} H U = \sum_{i=1}^r c_i \mu_i,$$

such that each $\mu_i$ term in the new Hamiltonian always acts
trivially, e.g., with an identity or a Pauli operator, on a set of
qubits. This allows tapering-off those qubits from the Hamiltonian.

For instance, consider the following Hamiltonian

$$H' = Z_0 X_1 - X_1 + Y_0 X_1,$$

where all terms in the Hamiltonian act on the second qubit with the $X$
operator. It is straightforward to show that each term in the
Hamiltonian commutes with $I_0 X_1$ and the ground-state eigenvector of
$H'$ is also an eigenvector of $I_0 X_1$ with eigenvalues $\pm 1$. We
can also rewrite the Hamiltonian as

$$H' = (Z_0 I_1 - I_0 I_1 + Y_0 I_1) I_0 X_1,$$

which gives us

$$H'|\psi \rangle = \pm1 (Z_0 I_1 - I_0 I_1 + Y_0 I_1)|\psi \rangle,$$

where $|\psi \rangle$ is an eigenvector of $H'$. This means that the
Hamiltonian $H$ can be simplified as

$$H_{tapered} = \pm1 (Z_0 - I_0 + Y_0).$$

The tapered Hamiltonian $H_{tapered}$ has the eigenvalues

$$[-2.41421, 0.41421],$$

and

$$[2.41421, -0.41421],$$

depending on the value of the $\pm 1$ prefactor. The eigenvalues of the
original Hamiltonian $H$ are

$$[2.41421, -2.41421,  0.41421, -0.41421],$$

which are thus reproduced by the tapered Hamiltonian.

More generally, we can construct the unitary $U$ such that each $\mu_i$
term acts with a Pauli-X operator on a set of qubits
$\left \{ j \right \}, j \in \left \{ l, ..., k \right \}$ where $j$ is
the qubit label. This guarantees that each term of the transformed
Hamiltonian commutes with each of the Pauli-X operators applied to the
$j$-th qubit:

$$[H', X^j] = 0,$$

and the eigenvectors of the transformed Hamiltonian $H'$ are also
eigenvectors of each of the $X^{j}$ operators. Then we can factor out
all of the $X^{j}$ operators from the transformed Hamiltonian and
replace them with their eigenvalues $\pm 1$. This gives us a set of
tapered Hamiltonians depending on which eigenvalue $\pm 1$ we chose for
each of the $X^{j}$ operators. For instance, in the case of two tapered
qubits, we have four eigenvalue sectors: $[+1, +1]$, $[-1, +1]$,
$[+1, -1]$, $[-1, -1]$. In these tapered Hamiltonians, the set of
$\left \{ j \right \}, j \in \left \{ l, ..., k \right \}$ qubits are
eliminated. For tapered molecular Hamiltonians, it is possible to
determine the optimal sector of the eigenvalues that corresponds to the
ground state. This is explained in more detail in the following
sections.

The unitary operator $U$ can be constructed as a
[Clifford](https://en.wikipedia.org/wiki/Clifford_gates) operator

$$U = \Pi_j \left [\frac{1}{\sqrt{2}} \left (X^{q(j)} + \tau_j \right) \right],$$

where $\tau$ denotes the generators of the symmetry group of $H$ and
$X^{q}$ operators act on those qubits that will be ultimately tapered
off from the Hamiltonian. The symmetry group of the Hamiltonian is
defined as an Abelian group of Pauli words that commute with each term
in the Hamiltonian (excluding $−I$). The
[generators](https://en.wikipedia.org/wiki/Generating_set_of_a_group) of
the symmetry group are those elements of the group that can be combined,
along with their inverses, to create any other member of the group.

Let\'s use the qubit tapering method and obtain the ground state energy
of the [Helium hydride
cation](https://en.wikipedia.org/wiki/Helium_hydride_ion)
$\textrm{HeH}^+$.

Tapering the molecular Hamiltonian
----------------------------------

In PennyLane, a
`molecular Hamiltonian <tutorial_quantum_chemistry>`{.interpreted-text
role="doc"} can be created by specifying the atomic symbols and
coordinates.


In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

import time

In [ ]:
%%time
# Molecular data
#qml.data.list_datasets()["qchem"]["NH3"]
data = qml.data.load("qchem", molname="BeH2", basis="STO-3G", bondlength=0.5)[0]
H = data.hamiltonian
print(H)

  (-5.5676479626047035) [I0]
+ (-0.5703531859821338) [Z12]
+ (-0.5703531859821338) [Z13]
+ (-0.3637095602693299) [Z11]
+ (-0.36370956026932977) [Z10]
+ (-0.057068495779112785) [Z6]
+ (-0.05706849577911276) [Z7]
+ (-0.05706849577909973) [Z9]
+ (-0.05706849577909962) [Z8]
+ (0.09812420390097004) [Z4]
+ (0.09812420390097026) [Z5]
+ (0.12079634345076504) [Z2]
+ (0.12079634345076518) [Z3]
+ (2.5752638603086124) [Z1]
+ (2.575263860308613) [Z0]
+ (0.08096415139941238) [Z10 Z12]
+ (0.08096415139941238) [Z11 Z13]
+ (0.08794357496969794) [Z4 Z6]
+ (0.08794357496969794) [Z5 Z7]
+ (0.087943574969742) [Z4 Z8]
+ (0.087943574969742) [Z5 Z9]
+ (0.08991824132125752) [Z6 Z12]
+ (0.08991824132125752) [Z7 Z13]
+ (0.08991824132130233) [Z8 Z12]
+ (0.08991824132130233) [Z9 Z13]
+ (0.092311618679811) [Z6 Z13]
+ (0.092311618679811) [Z7 Z12]
+ (0.09231161867985704) [Z8 Z13]
+ (0.09231161867985704) [Z9 Z12]
+ (0.09392738750330087) [Z2 Z4]
+ (0.09392738750330087) [Z3 Z5]
+ (0.09427773555497992) [Z6 Z8]
+ (0.09427

This Hamiltonian contains 27 terms where each term acts on up to four
qubits.

We can now obtain the symmetry generators and the $X^{j}$ operators that
are used to construct the unitary $U$ operator that transforms the
$\textrm{HeH}^+$ Hamiltonian. In PennyLane, these are constructed by
using the `~.pennylane.symmetry_generators`{.interpreted-text
role="func"} and `~.pennylane.paulix_ops`{.interpreted-text role="func"}
functions.


In [ ]:
qubits = len(H.wires)
generators = qml.symmetry_generators(H)
paulixops = qml.paulix_ops(generators, qubits)

for idx, generator in enumerate(generators):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops[idx]}")

generator 1:   (1.0) [Z6 Z7], paulix_op: PauliX(wires=[7])
generator 2:   (1.0) [Z8 Z9], paulix_op: PauliX(wires=[9])
generator 3:   (1.0) [Z0 Z1 Z4 Z5 Z10 Z11], paulix_op: PauliX(wires=[11])
generator 4:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10 Z12], paulix_op: PauliX(wires=[12])
generator 5:   (1.0) [Z0 Z3 Z4 Z6 Z8 Z10 Z13], paulix_op: PauliX(wires=[13])


/usr/local/lib/python3.8/dist-packages/pennylane/operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pennylane/operation.py:1993: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


Once the operator $U$ is applied, each of the Hamiltonian terms will act
on the qubits $q_2, q_3$ either with the identity or with a Pauli-X
operator. For each of these qubits, we can simply replace the Pauli-X
operator with one of its eigenvalues $+1$ or $-1$. This results in a
total number of $2^k$ Hamiltonians, where $k$ is the number of
tapered-off qubits and each Hamiltonian corresponds to one eigenvalue
sector. The optimal sector corresponding to the ground-state energy of
the molecule can be obtained by using the
`~.pennylane.qchem.optimal_sector`{.interpreted-text role="func"}
function.


In [ ]:
#n_electrons = 2
#paulix_sector = qml.qchem.optimal_sector(H, generators, n_electrons)
paulix_sector = data.optimal_sector
print(paulix_sector)

[1, 1, 1, -1, -1]


The optimal eigenvalues are $-1, -1$ for qubits $q_2, q_3$,
respectively. We can now build the tapered Hamiltonian with the
`~.pennylane.taper`{.interpreted-text role="func"} function which
constructs the operator $U$, applies it to the Hamiltonian and finally
tapers off the qubits $q_2, q_3$ by replacing the Pauli-X operators
acting on those qubits with the optimal eigenvalues.


In [ ]:
from tqdm.notebook import tqdm
from IPython.display import display
from pennylane.qchem.tapering import clifford, _observable_mult, simplify

In [ ]:
def taper(h, generators, paulixops, paulix_sector):
    print("clifford")
    u = clifford(generators, paulixops)
    print("my_observable_mult") # TODO: describe step
    h = my_observable_mult(my_observable_mult(u, h), u)
    val = np.ones(len(h.terms()[0])) * complex(1.0)
    wireset = u.wires + h.wires
    print("wiremap") # TODO: describe step
    wiremap = dict(zip(wireset, range(len(wireset) + 1)))
    paulix_wires = [x.wires[0] for x in paulixops]
    for idx, w in enumerate(paulix_wires):
        for i in range(len(h.terms()[0])):
            #print("1. pauli:", h.terms()[1][i]) 
            s = qml.pauli.pauli_word_to_string(h.terms()[1][i], wire_map=wiremap)
            if s[w] == "X":
                val[i] *= paulix_sector[idx]
    o = []
    wires_tap = [i for i in h.wires if i not in paulix_wires]
    wiremap_tap = dict(zip(wires_tap, range(len(wires_tap) + 1)))
    for i in range(len(h.terms()[0])):
        #print("2. pauli:", h.terms()[1][i]) 
        s = qml.pauli.pauli_word_to_string(h.terms()[1][i], wire_map=wiremap)
        wires = [x for x in h.wires if x not in paulix_wires]
        o.append(
            qml.pauli.string_to_pauli_word(
                "".join([s[wiremap[i]] for i in wires]), wire_map=wiremap_tap
            )
        )
    c = np.multiply(val, h.terms()[0])
    c = qml.math.stack(c)
    tapered_ham = simplify(qml.Hamiltonian(c, o))
    # If simplified Hamiltonian is missing wires, then add wires manually for consistency
    if wires_tap != list(tapered_ham.wires):
        identity_op = functools.reduce(
            lambda i, j: i @ j,
            [
                qml.Identity(wire)
                for wire in Wires.unique_wires([tapered_ham.wires, Wires(wires_tap)])
            ],
        )
        tapered_ham = qml.Hamiltonian(
            np.array([*tapered_ham.coeffs, 0.0]), [*tapered_ham.ops, identity_op]
        )
    return tapered_ham

In [ ]:
def my_observable_mult(obs_a, obs_b):
    o = []
    c = []

    for i in tqdm(range(len(obs_a.terms()[0])), desc="obs mult", leave=False):
        for j in range(len(obs_b.terms()[0])):
            op, phase = qml.pauli.pauli_mult_with_phase(obs_a.terms()[1][i], obs_b.terms()[1][j])
            o.append(op)
            c.append(phase * obs_a.terms()[0][i] * obs_b.terms()[0][j])
    print("obs simplify")
    return my_simplify(qml.Hamiltonian(qml.math.stack(c), o))

In [ ]:
def my_simplify(h, cutoff=1.0e-12):
    wiremap = dict(zip(h.wires, range(len(h.wires) + 1)))

    c, o = [], []
    display("count ops:", len(h.ops)) # 88704
    for i, op in tqdm(enumerate(h.ops), desc="simplify", leave=False):
        op = qml.operation.Tensor(op).prune()
        op = qml.pauli.pauli_word_to_string(op, wire_map=wiremap)
        if op not in o:
            c.append(h.coeffs[i])
            o.append(op)
        else:
            c[o.index(op)] += h.coeffs[i]

    coeffs, ops = [], []
    display("nonzero_ind") 
    nonzero_ind = np.argwhere(abs(np.array(c)) > cutoff).flatten()
    for i in nonzero_ind:
        coeffs.append(c[i])
        ops.append(qml.pauli.string_to_pauli_word(o[i], wire_map=wiremap))

    try:
        coeffs = qml.math.stack(coeffs)
    except ValueError:
        pass

    return qml.Hamiltonian(np.array(coeffs), ops)

### TODO: instead of calculating tapered Hamiltonian transform `dict` object to `qml.Hamiltonian`

Example:

```
{'terms': {'IIIIIIIII': tensor(-5.34271842+0.j, requires_grad=True), 'ZIIIIIIII': tensor(2.57526386+0.j, requires_grad=True), ...}
```

should be changed to:

```
((-5.342718417485251+0j)) [I0]
+ ((-0.3637095602693296+0j)) [Z10]
+ ((-0.11413699155822549+0j)) [Z6]
+ ...
```

because `qml.utils.sparse_hamiltonian` requires `qml.Hamiltonian` format.


In [ ]:
# TODO: To be transformed to qml.Hamiltonian
H_tapered = data.tapered_hamiltonian
print(H_tapered)

{'terms': {'IIIIIIIII': tensor(-5.34271842+0.j, requires_grad=True), 'ZIIIIIIII': tensor(2.57526386+0.j, requires_grad=True), 'IZIIIIIII': tensor(2.57526386+0.j, requires_grad=True), 'ZZIIIIIII': tensor(0.55434555+0.j, requires_grad=True), 'YZZZYIIII': tensor(0.05996012+0.j, requires_grad=True), 'YIZZYIIII': tensor(0.07634839+0.j, requires_grad=True), 'XZZZXIIII': tensor(0.05996012+0.j, requires_grad=True), 'XIZZXIIII': tensor(0.07634839+0.j, requires_grad=True), 'YZZZZZIIY': tensor(0.07224548+0.j, requires_grad=True), 'YIZZZZIIY': tensor(0.07833438+0.j, requires_grad=True), 'XZZZZZIIX': tensor(0.07224548+0.j, requires_grad=True), 'XIZZZZIIX': tensor(0.07833438+0.j, requires_grad=True), 'YXXYIIIII': tensor(0.00707583+0.j, requires_grad=True), 'YYXXIIIII': tensor(-0.00707583+0.j, requires_grad=True), 'XXYYIIIII': tensor(-0.00707583+0.j, requires_grad=True), 'XYYXIIIII': tensor(0.00707583+0.j, requires_grad=True), 'YYZXZIZZZ': tensor(0.00430252+0.j, requires_grad=True), 'YXZYZIZZZ': tens

In [ ]:
%%time
#  simplify number of operations: 21312 + 681984
H_tapered = taper(H, generators, paulixops, paulix_sector)
print(H_tapered)

clifford
my_observable_mult


obs mult:   0%|          | 0/32 [00:00<?, ?it/s]

obs simplify


'count ops:'

21312

...

'count ops:'

681984

simplify: 0it [00:00, ?it/s]

'nonzero_ind'

wiremap
  ((-5.342718417485251+0j)) [I0]
+ ((-0.3637095602693296+0j)) [Z10]
+ ((-0.11413699155822549+0j)) [Z6]
+ ((-0.11413699155819929+0j)) [Z8]
+ ((-0.03250606633603204+0j)) [X5]
+ ((0.002393377358553489+0j)) [X6]
+ ((0.0023933773585547245+0j)) [X8]
+ ((0.0981242039009699+0j)) [Z4]
+ ((0.0981242039009702+0j)) [Z5]
+ ((0.12079634345076498+0j)) [Z2]
+ ((0.12079634345076508+0j)) [Z3]
+ ((2.57526386030861+0j)) [Z0]
+ ((2.57526386030861+0j)) [Z1]
+ ((-0.05026552406931751+0j)) [Y2 Y3]
+ ((-0.045653397313168025+0j)) [X5 Z10]
+ ((-0.023992822113378085+0j)) [Y4 Y10]
+ ((-0.023992822113378085+0j)) [X4 X10]
+ ((-0.011036750428208396+0j)) [Y3 Y4]
+ ((-0.008006848277019947+0j)) [X5 X10]
+ ((-0.007954091028334408+0j)) [Y4 Y5]
+ ((-0.006547316135426807+0j)) [Y1 Y3]
+ ((-0.00297399834826594+0j)) [Y0 Y1]
+ ((-0.0017250747625590782+0j)) [Y1 Y2]
+ ((0.003089361863779837+0j)) [Y1 Y4]
+ ((0.0054929461055843705+0j)) [X6 X10]
+ ((0.0054929461055843705+0j)) [Y6 Y10]
+ ((0.005492946105587151+0j)) [X8 X10]
+ 

The new Hamiltonian has only 9 non-zero terms acting on only 2 qubits!
We can verify that the original and the tapered Hamiltonian both give
the correct ground state energy of the $\textrm{HeH}^+$ cation, which is
$-2.862595242378$ Ha computed with the full configuration interaction
(FCI) method. In PennyLane, it\'s possible to build a sparse matrix
representation of Hamiltonians. This allows us to directly diagonalize
them to obtain exact values of the ground-state energies.


In [ ]:
H_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H), wires=H.wires)
H_tapered_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H_tapered), wires=H_tapered.wires)

print("Eigenvalues of H:\n", qml.eigvals(H_sparse, k=16))
print("\nEigenvalues of H_tapered:\n", qml.eigvals(H_tapered_sparse, k=4))

In [ ]:
data.fci_spectrum

array([-13.68996939, -13.5791884 , -13.5791884 , -13.5791884 ,
       -13.5791884 , -13.44151903, -13.49655931, -13.49655931,
       -13.44151903, -13.44151903, -13.49655931, -13.49655931,
       -13.38005202, -13.38005202, -13.44151903, -13.49655931,
       -13.49655931, -13.41282735, -13.41282735, -13.41282735,
       -13.41282735, -13.41282735, -13.41282735, -13.34679104,
       -13.34679104, -13.34679104, -13.34679104])

Tapering the reference state
============================

The ground state Hartree-Fock energy of $\textrm{HeH}^+$ can be computed
by directly applying the Hamiltonians to the Hartree-Fock state. For the
tapered Hamiltonian, this requires transforming the Hartree-Fock state
with the same symmetries obtained for the original Hamiltonian. This
reduces the number of qubits in the Hartree-Fock state to match that of
the tapered Hamiltonian. It can be done with the
`~.pennylane.qchem.taper_hf`{.interpreted-text role="func"} function.


In [ ]:
# DEBUG: function taper_hf returns different HF-state than from dataset
#state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
#                                   num_electrons=n_electrons, num_wires=len(H.wires))
state_tapered = data.tapered_hf_state
print(state_tapered)

[1 1 1 1 1 1 0 0 0]


Recall that the original Hartree-Fock state for the $\textrm{HeH}^+$
cation is $[1 1 0 0]$. We can now generate the qubit representation of
these states and compute the Hartree-Fock energies for each Hamiltonian.


In [ ]:
from pennylane import qchem
# DEBUG: function taper_hf returns different HF-state than from dataset
#hf_state = qchem.hf_state(n_electrons, len(H.wires)) 
#hf_state

tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], requires_grad=True)

In [ ]:
hf_state = data.hf_state
hf_state

tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], requires_grad=True)

In [ ]:
%%time
dev = qml.device("default.qubit", wires=H.wires)
@qml.qnode(dev)
def circuit():
    qml.BasisState(hf_state, wires=H.wires)
    return qml.state()

qubit_state = circuit()
HF_energy = qubit_state.T @ qml.utils.sparse_hamiltonian(H).toarray() @ qubit_state
print(f"HF energy: {np.real(HF_energy):.8f} Ha")

dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def circuit():
    # TODO: Basis state wires after tappering
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    return qml.state()

qubit_state = circuit()
HF_energy = qubit_state.T @ qml.utils.sparse_hamiltonian(H_tapered).toarray() @ qubit_state
print(f"HF energy (tapered): {np.real(HF_energy):.8f} Ha")

HF energy: -13.64614980 Ha
HF energy (tapered): -13.64614980 Ha
CPU times: user 4.77 s, sys: 4.4 s, total: 9.17 s
Wall time: 9.84 s


These values are identical to the reference Hartree-Fock energy
$-2.8543686493$ Ha.

VQE simulation
==============

Compare with data from: [Tapered observables data](https://pennylane.ai/qml/datasets_qchem.html#tapered-observables-data)

See also: [Givens rotations for quantum chemistry](https://pennylane.ai/qml/demos/tutorial_givens_rotations.html)

Finally, we can use the tapered Hamiltonian and the tapered reference
state to perform a VQE simulation and compute the ground-state energy of
the $\textrm{HeH}^+$ cation. We build a tapered variational ansatz
[\[3\]](https://pennylane.ai/qml/demos/tutorial_givens_rotations.html)
that prepares an entangled state by evolving the tapered Hartree-Fock
state using the tapered particle-conserving gates, i.e., the
`~.pennylane.SingleExcitation`{.interpreted-text role="func"} and
`~.pennylane.DoubleExcitation`{.interpreted-text role="func"} operations
tapered using `~.pennylane.qchem.taper_operation`{.interpreted-text
role="func"}.


In [ ]:
%%time
singles, doubles = qml.qchem.excitations(n_electrons, len(H.wires))
tapered_doubles = [
    qml.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=double) for double in doubles
]
tapered_singles = [
    qml.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=single) for single in singles
]

dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)

CPU times: user 1min 23s, sys: 1.09 s, total: 1min 24s
Wall time: 1min 25s


We define an optimizer and the initial values of the circuit parameters
and optimize the circuit parameters with respect to the ground state
energy.


In [ ]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(doubles) + len(singles), requires_grad=True)

for n in range(1, 41):
    params, energy = optimizer.step_and_cost(tapered_circuit, params)
    if not n % 5:
        #print(f"n: {n}, E: {energy:.9f} Ha, Params: {params}")
        print(f"n: {n}, E: {energy:.9f} Ha")

n: 5, E: -13.471570696 Ha
n: 10, E: -8.696350402 Ha
n: 15, E: -12.978065634 Ha
n: 20, E: -13.639642031 Ha
n: 25, E: -13.646374973 Ha
n: 30, E: -13.646460033 Ha
n: 35, E: -13.646465901 Ha
n: 40, E: -13.646470961 Ha


The computed energy matches the FCI energy, $-2.862595242378$ Ha, while
the number of qubits and the number of Hamiltonian terms are
significantly reduced with respect to their original values.

Conclusions
===========

Molecular Hamiltonians possess symmetries that can be leveraged to
reduce the number of qubits required in quantum computing simulations.
This tutorial introduces a PennyLane functionality that can be used for
qubit tapering based on $\mathbb{Z}_2$ symmetries. The procedure
includes obtaining tapered Hamiltonians and tapered reference states
that can be used in variational quantum algorithms such as VQE.

References
==========

About the author
================
